## Environment Setup Test

Run the cell below to validate that you have properly configured your virtual environment

In [1]:
import logging
import os
import pandas as pd
import string
import random

#for UI
import streamlit as st
from dotenv import load_dotenv
from langchain.callbacks import StdOutCallbackHandler
from PIL import Image

# for PDF Download 
import tempfile
import pickle

from langchain.document_loaders import PyPDFLoader
from langchain.text_splitter import RecursiveCharacterTextSplitter

#for IBM embedding model
from ibm_watsonx_ai.metanames import EmbedTextParamsMetaNames
from langchain_ibm import WatsonxEmbeddings
from ibm_watsonx_ai.client import APIClient
from ibm_watsonx_ai.foundation_models import Model

# for Milvus 
from pymilvus import connections, utility, Collection, CollectionSchema, FieldSchema,DataType

# for LLM model
from ibm_watsonx_ai.foundation_models import Model

### Errors above or not?
If no errors appear when executing the cell above, then proceed to the next steps in the workshop material.  If errors appear, return to the environment setup screen to ensure you have:
1. Install all required libraries as described in that section
2. Activated your virtual environment

# Check LLM connection

In [2]:
load_dotenv()
api_key =           os.getenv("WATSONX_APIKEY",         None)
ibm_cloud_url =     os.getenv("IBM_CLOUD_URL",          None)
project_id =        os.getenv("PROJECT_ID",             None)

In [3]:
model_id_llm='meta-llama/llama-3-8b-instruct'
creds = {
        "url": ibm_cloud_url,
        "apikey": api_key 
    }

In [4]:
model_llm = Model(
    model_id = model_id_llm,
    params = {
        'decoding_method': "greedy",
        'min_new_tokens': 1,
        'max_new_tokens': 400,
        'temperature': 0.0,
        'repetition_penalty': 1
    },
    credentials=creds,
    project_id=project_id)

In [5]:
prompt = """You are a helpful assistance that would answer user's question with simple explanation. You will be given questions about general knowledge, please answer them shortly and politely
question: What is IBM?
answer: """
response = model_llm.generate_text(prompt)
print(response)

 IBM (International Business Machines) is a multinational technology company that provides hardware, software, and services. It is one of the largest and most well-known technology companies in the world. IBM is known for its computers, servers, and software, as well as its consulting and outsourcing services. It was founded in 1911 and is headquartered in Armonk, New York. IBM is often referred to as "Big Blue" due to its blue logo and its large size in the technology industry.


# Check Milvus connection

In [8]:
host =          str(os.getenv("MILVUS_HOST",            None))
port =              os.getenv("MILVUS_PORT",            None)
server_pem_path =   os.getenv("MILVUS_SERVER_PEM_PATH", None)
server_name =       os.getenv("MILVUS_SERVER_NAME",     None)
user =              os.getenv("MILVUS_USER",            None)
password =          os.getenv("MILVUS_PASSWORD",        None)

In [9]:
print('connecting to milvus...')
connections.connect(
    "default", 
    host = host, 
    port = port, 
    secure=True, 
    server_pem_path = server_pem_path,
    server_name = server_name,
    user = user,
    password=password)
print("Milvus connected")

connecting to milvus...
Milvus connected


# Check Embedding model connection

In [10]:
emb_api_key =       os.getenv("EMB_WATSONX_APIKEY",     None)
emb_ibm_cloud_url = os.getenv("EMB_IBM_CLOUD_URL",      None)
emb_project_id =    os.getenv("EMB_PROJECT_ID",         None)
emb_space_id =      os.getenv("EMB_SPACE_ID",           None)
deployment_id =     os.getenv("EMB_DEPLOYMENT_ID",      None)

In [11]:
emb_creds = {
        "url": emb_ibm_cloud_url,
        "apikey": emb_api_key 
    }

client = APIClient(credentials=emb_creds, space_id=emb_space_id)

In [12]:
chunks = ['เขาชอบไปเดินเล่นที่สวนสาธารณะในวันหยุด','วันนี้อากาศร้อนมาก']
payload = {
    'input_data': [
        {
            'values': [
                chunks
            ]
        }
    ]
}   
reponse = client.deployments.score(deployment_id, payload)
print(reponse)

{'predictions': [{'fields': ['sentence', 'embedding'], 'values': [['เขาชอบไปเดินเล่นที่สวนสาธารณะในวันหยุด', [-0.2392904907464981, -0.189870223402977, -0.57694011926651, 1.023754596710205, 0.3908231258392334, 1.6365150213241577, -0.13832838833332062, 0.5410310626029968, 0.3928060531616211, -0.5711298584938049, 0.5849242806434631, 1.5124703645706177, 0.0019164839759469032, 0.6231353878974915, -0.44601020216941833, 0.40461212396621704, 0.9897261261940002, 0.809762179851532, -0.9995424747467041, 0.27537602186203003, -0.6118722558021545, -0.7498077750205994, -0.8579646944999695, 1.5173606872558594, -0.560527503490448, -0.3529303967952728, 1.360764980316162, -0.6861627101898193, -1.9828771352767944, 0.07640975713729858, 0.3546256721019745, 1.0148214101791382, 0.491934210062027, -0.6217899918556213, -0.09395813196897507, 0.0084333885461092, 0.04894046485424042, -0.2933293282985687, 1.2188050746917725, 0.3738785982131958, 0.7557473182678223, -0.7077399492263794, -1.1205154657363892, 0.5246590